In [133]:
import ROOT
from ROOT import gStyle
import numpy as np
import ctypes
import os
from IPython.display import display 
import pandas as pd 

samples=os.listdir("SR_OS")
if "MJ.root" in samples:
    os.system("rm SR_OS/MJ.root")

def biner(edges,bin_widths,histogram):
    if (len(edges)+1!=len(bin_widths)):
        print("Check edges and bin widths array sizes!")
        return
    
    bins=[]
    first_bin = histogram.GetXaxis().GetBinLowEdge(1)
    #print(first_bin)
    last_bin = histogram.GetXaxis().GetBinUpEdge(histogram.GetNbinsX())
    #print(last_bin)
    for i in range(0,len(edges)):
        n_spaces = int((edges[i] - first_bin)/bin_widths[i])
        bins = np.concatenate((bins,np.linspace(first_bin,edges[i],n_spaces,endpoint=False)))
        first_bin = edges[i]
        if edges[i]==edges[-1]:
            n_spaces = int((last_bin - edges[i])/bin_widths[i+1])
            bins = np.concatenate((bins,np.linspace(edges[i],last_bin,n_spaces,endpoint=False)))
            bins = np.concatenate((bins,[last_bin]))
    return bins

def normalization(hist_list,norm_bin):
    for hist in hist_list:
        for i in range(1,hist.GetNbinsX()+1):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=hist.GetBinWidth(i)/norm_bin
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)
            

In [134]:
histos_1={
"lep+nu_pt_topo":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau)(GeV)(i)",1.2,200],
"tau+nu_pt_topo":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau)(GeV)(i)",1.2,200],
"tau_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_inside":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau)(GeV)(i)",1.2,200,45,1000],
"tau_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_outside":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau)(GeV)(o)",1.2,200,45,1000],
"lep_pt0_topo_dphi_btag_iso_rnn_ptmu_omega_mreco":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#mu)(GeV)",1.2,200],
"sum_pt_pt_topo":["yy",[[180.0],[5,20],5],"Events/5 GeV","#sum p_{T}(GeV)",1.2,400],
"MET_topo_dphi_btag_iso_rnn_ptmu_omega_mreco":["yy",[[120.0],[5,10],5],"Events/5 GeV","MET(GeV)",1.2,300],
"Z_pt_cuts_inside":["yy",[[180.0],[10,20],10],"Events/5 GeV","p_{T}(Z)(GeV)(i)",1.2,400],
"Z_pt_cuts_outside":["yy",[[180.0],[10,20],10],"Events/5 GeV","p_{T}(Z)(GeV)(o)",1.2,400],
"delta_phi_cuts_butphi":["n",[1,1,1],"Events/0.1","#Delta#varphi(#tau,#mu)",1.4,2.3,1.0,3.0],
"delta_phi_cuts":["n",[1,1,1],"Events/0.1","#Delta#varphi(#tau,#mu)",1.4,1.0],
"ratio_ptjet_zpt_cuts":["n",[1,1,1],"Events/0.1","#Delta#varphi(#tau,#mu)",1.4,2.5],
"ratio_lpt_tpt_cuts":["n",[1,1,1],"Events/0.1","#Delta#varphi(#tau,#mu)",1.4,2.5],
"n_tracks":["n",[1,1,1],"Events/1","NTracks",1.4,30],
"n_tracks_cuts":["n",[1,1,1],"Events/1","NTracks",1.4,30],
"jet_n_topo_dphi_btag_iso_rnn_ptmu_omega_mreco":["n",[1,1,1],"Events/1","n-jets",1.4,9],
"jet_n_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt":["n",[1,1,1],"Events/1","n-jets",1.4,9],
"ljet1_pt_topo_cuts":["yy",[[120.0],[5,10],5],"Events/5 GeV","p_{T}(jet1)(GeV)",1.2,200],
"ljet1_pt_topo_cuts_tpt":["yy",[[120.0],[5,10],5],"Events/5 GeV","p_{T}(jet1)(GeV)",1.2,200],
"ljet2_pt_topo_cuts":["yy",[[120.0],[5,10],5],"Events/5 GeV","p_{T}(jet2)(GeV)",1.2,200],
"ljet2_pt_topo_cuts_tpt":["yy",[[120.0],[5,10],5],"Events/5 GeV","p_{T}(jet2)(GeV)",1.2,200],

"lep+nu_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau)(GeV)(i)",1.2,200],
"tau+nu_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau)(GeV)(i)",1.2,200],
"lep+nu_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau)(GeV)(i)",1.2,200],
"tau+nu_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau)(GeV)(i)",1.2,200],
"h_reco_mass_topo_dphi_btag_iso_rnn_ptmu_omega_tpt":["yy",[[60,80,100,120.0],[20,10,5,10,20],5],"Events/5 GeV","m_{reco}(GeV)(i)",1.2,240,60,120],
"reco_mass_met_outside_topo_dphi_btag_iso_rnn_ptmu_omega_tpt":["yy",[[60,80,100,120.0],[20,10,5,10,20],5],"Events/5 GeV","m_{reco}(GeV)(o)",1.2,240,60,120],
"tau_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt_inside":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau)(GeV)(i)",1.2,200,45,1000],
"tau_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt_outside":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau)(GeV)(o)",1.2,200,45,1000],
"tau+nu_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau+#nu)(GeV)",1.2,200],
"tau+nu_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau+#nu)(GeV)",1.2,200],
"sum_pt_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco":["yy",[[180.0],[10,20],10],"Events/5 GeV","#sum p_{T}(GeV)",1.2,400],
"lep_pt0_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt":["yy",[[120.0],[5,10],5],"Events/5 GeV","p_{T}(#mu)(GeV)",1.2,200],
"MET_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt":["yy",[[120.0],[5,10],5],"Events/5 GeV","MET(GeV)",1.2,300],
"Z_pt_cuts_tpt_inside":["yy",[[180.0],[10,20],10],"Events/5 GeV","p_{T}(Z)(GeV)(i)",1.2,400],
"Z_pt_cuts_tpt_outside":["yy",[[180.0],[10,20],10],"Events/5 GeV","p_{T}(Z)(GeV)(o)",1.2,400],
"transverse_lepton_mass_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt":["yy",[[120.0],[5,10],5],"Events/5 GeV","m_{T}(#mu)(GeV)",1.2,100],
"omega_topo_dphi_btag_iso_rnn_ptmu_mreco_tpt":["yy",[[-0.2,1.6],[0.4,0.2,0.4],0.2],"Events/0.2","#Omega",1.2,3,-0.2,1.6],
"delta_phi_cuts_tpt":["n",[1,1,1],"Events/0.1","#Delta#varphi(#tau,#mu)",1.4,2.3],
"rnn_score_1prong_topo_dphi_btag_iso_ptmu_omega_mreco_tpt":["yy",[[0.8],[0.1,0.0999],0.1],"Events/0.1","jetRNNScore",1.4,1,0.4,1000],
"rnn_score_3prong_topo_dphi_btag_iso_ptmu_omega_mreco_tpt":["yy",[[0.8],[0.1,0.0999],0.1],"Events/0.1","jetRNNScore",1.4,1,0.5,1000],
"ratio_ptjet_zpt_cuts_tpt":["n",[1,1,1],"Events/0.1","r_{pT(j)/pT(Z)}",1.4,3],
"ratio_lpt_tpt_cuts_tpt":["n",[1,1,1],"Events/0.1","r_{pT(#mu)/pT(#tau)}",1.4,3],
"tau_nprongs_cuts":["n",[1,1,1],"Events/1","n-prongs",1.4,4],
"tau_nprongs_cuts_tpt":["n",[1,1,1],"Events/1","n-prongs",1.4,4],
    
"lep_pt_1p_cuts":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(l 1p)(GeV)",1.2,200],
"lep_pt_3p_cuts":["yy",[[65,125.0],[5,10,25],5],"Events/10 GeV","p_{T}(l 3p)(GeV)",1.2,200],
"lep_pt_1p_cuts_tpt":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(l 1p)(GeV)",1.2,200],
"lep_pt_3p_cuts_tpt":["yy",[[65,125.0],[5,10,25],5],"Events/10 GeV","p_{T}(l 3p)(GeV)",1.2,200],
    
"lep_eta_1p_cuts":["yy",[[-1.0,-0.1,0.1,1.0],[0.5,0.3,0.2,0.3,0.5],0.3],"Events/0.2 ","#eta(l 1p)",1.2,2.5],
"lep_eta_3p_cuts":["yy",[[-1.0,-0.1,0.1,1.0],[0.5,0.3,0.2,0.3,0.5],0.3],"Events/0.5 ","#eta(l 1p)",1.2,2.5],
"lep_eta_1p_cuts_tpt":["yy",[[-1.0,-0.1,0.1,1.0],[0.5,0.3,0.2,0.3,0.5],0.3],"Events/0.2 ","#eta(l 1p)",1.2,2.5],
"lep_eta_3p_cuts_tpt":["yy",[[-1.0,-0.1,0.1,1.0],[0.5,0.3,0.2,0.3,0.5],0.3],"Events/0.5 ","#eta(l 3p)",1.2,2.5],

}

histos_2 = {
"tau_pt_1p_cuts_1D":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau 1p)(GeV)",1.2,200,45,1000],
"tau_pt_3p_cuts_1D":["yy",[[65,125.0],[5,10,25],5],"Events/10 GeV","p_{T}(#tau 3p)(GeV)",1.2,200,45,1000],

"tau_pt_1p_cuts_tpt_1D":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau 1p)(GeV)",1.2,200],
"tau_pt_3p_cuts_tpt_1D":["yy",[[65,125.0],[5,10,25],5],"Events/10 GeV","p_{T}(#tau 3p)(GeV)",1.2,200],


"tau_eta_1p_cuts_1D":["yy",[[-1.0,-0.1,0.1,1.0],[0.5,0.3,0.2,0.3,0.5],0.3],"Events/0.2 ","#eta(#tau 1p)",1.2,2.5],
"tau_eta_3p_cuts_1D":["yy",[[-1.0,-0.1,0.1,1.0],[0.5,0.3,0.2,0.3,0.5],0.3],"Events/0.5 ","#eta}(#tau 3p)",1.2,2.5],

"tau_eta_1p_cuts_tpt_1D":["yy",[[-1.0,-0.1,0.1,1.0],[0.5,0.3,0.2,0.3,0.5],0.3],"Events/0.2 ","#eta(#tau 1p)",1.2,2.5],
"tau_eta_3p_cuts_tpt_1D":["yy",[[-1.0,-0.1,0.1,1.0],[0.5,0.3,0.2,0.3,0.5],0.3],"Events/0.5 ","#eta(#tau 3p)",1.2,2.5],

}

histos = {}
histos.update(histos_1)
histos.update(histos_2)


histos_2d = {
"tau_pt_1p_cuts":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau 1p)(GeV)",1.2,200,45,1000],
"tau_pt_3p_cuts":["yy",[[65,125.0],[5,10,25],5],"Events/10 GeV","p_{T}(#tau 3p)(GeV)",1.2,200,45,1000],

"tau_pt_1p_cuts_tpt":["yy",[[65,125.0],[5,10,25],5],"Events/5 GeV","p_{T}(#tau 1p)(GeV)",1.2,200],
"tau_pt_3p_cuts_tpt":["yy",[[65,125.0],[5,10,25],5],"Events/10 GeV","p_{T}(#tau 3p)(GeV)",1.2,200],


"tau_eta_1p_cuts":["yy",[[0.0],[0.1,0.1],0.1],"Events/0.2 ","#eta(#tau 1p)",1.2,2.5],
"tau_eta_3p_cuts":["yy",[[0.0],[0.1,0.1],0.1],"Events/0.5 ","#eta}(#tau 3p)",1.2,2.5],

"tau_eta_1p_cuts_tpt":["yy",[[0.0],[0.1,0.1],0.1],"Events/0.2 ","#eta(#tau 1p)",1.2,2.5],
"tau_eta_3p_cuts_tpt":["yy",[[0.0],[0.1,0.1],0.1],"Events/0.5 ","#eta(#tau 3p)",1.2,2.5],

}

for i in ["SR_OS/","SR_SS/","CR_OS/","CR_SS/"]:
    
    print(os.getcwd())
    os.chdir(i)
    files = [l for l in os.listdir() if (".root" in l and "MJ" not in l)]
    for f in files:
        for h in histos_2d:
            file=ROOT.TFile.Open(f,"update")
            histo = file.Get(h).ProjectionX(h+"_1D",1,3)
            histo.Write("",ROOT.TObject.kOverwrite)
            file.Close()
    os.chdir("..")


/home/diego/HEP/TauID/TauMu/Plots
/home/diego/HEP/TauID/TauMu/Plots
/home/diego/HEP/TauID/TauMu/Plots
/home/diego/HEP/TauID/TauMu/Plots


In [144]:
##### SELECTING GENERATOR FOR SIGNAL SAMPLE
signal_sample="Signal_PoPy.root"
signal_string=signal_sample.strip("Signal").strip(".root")
production=True

#### CREATING THE CORRESPONDING MC.ROOT FILE EVERYTIME YOU CHANGE THE SIGNAL SAMPLE

for i in ["SR_OS/","SR_SS/","CR_OS/","CR_SS/"]:
    os.system("rm "+i+"MC.root")
    os.chdir(i)
    os.system("hadd MC.root "+"BG.root "+signal_sample)
    
    files = [i for i in os.listdir() if ".root" in i]
    
    os.chdir("..")

signal_sample_file=ROOT.TFile.Open("SR_OS/"+signal_sample,"READ")

tau_matched_cuts_1p=signal_sample_file.Get("tau_matched_1p_topo_dphi_btag_iso_rnn_ptmu_omega_mreco")
tau_matched_cuts_3p=signal_sample_file.Get("tau_matched_3p_topo_dphi_btag_iso_rnn_ptmu_omega_mreco")
tau_matched_cuts_tpt_1p=signal_sample_file.Get("tau_matched_1p_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt")
tau_matched_cuts_tpt_3p=signal_sample_file.Get("tau_matched_3p_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt")

#tau_matched_cuts_1p=signal_sample_file.Get("tau_matched_cuts_1p")
#tau_matched_cuts_3p=signal_sample_file.Get("tau_matched_cuts_3p")
#tau_matched_cuts_tpt_1p=signal_sample_file.Get("tau_matched_cuts_tpt_1p")
#tau_matched_cuts_tpt_3p=signal_sample_file.Get("tau_matched_cuts_tpt_3p")


tau_matched_cuts_1p.SetDirectory(0)
tau_matched_cuts_3p.SetDirectory(0)
tau_matched_cuts_tpt_1p.SetDirectory(0)
tau_matched_cuts_tpt_3p.SetDirectory(0)

signal_sample_file.Close()

#1 PRONG INFO

tau_truth_cuts_1p=tau_matched_cuts_1p.GetBinContent(2)
tau_truth_cuts_1p_error=tau_matched_cuts_1p.GetBinError(2)
tau_fake_cuts_1p=tau_matched_cuts_1p.GetBinContent(1)
tau_fake_cuts_1p_error=tau_matched_cuts_1p.GetBinError(1)

tau_truth_cuts_tpt_1p=tau_matched_cuts_tpt_1p.GetBinContent(2)
tau_truth_cuts_tpt_1p_error=tau_matched_cuts_tpt_1p.GetBinError(2)
tau_fake_cuts_tpt_1p=tau_matched_cuts_tpt_1p.GetBinContent(1)
tau_fake_cuts_tpt_1p_error=tau_matched_cuts_tpt_1p.GetBinError(1)

#3 PRONG INFO

tau_truth_cuts_3p=tau_matched_cuts_3p.GetBinContent(2)
tau_truth_cuts_3p_error=tau_matched_cuts_3p.GetBinError(2)
tau_fake_cuts_3p=tau_matched_cuts_3p.GetBinContent(1)
tau_fake_cuts_3p_error=tau_matched_cuts_3p.GetBinError(1)

tau_truth_cuts_tpt_3p=tau_matched_cuts_tpt_3p.GetBinContent(2)
tau_truth_cuts_tpt_3p_error=tau_matched_cuts_tpt_3p.GetBinError(2)
tau_fake_cuts_tpt_3p=tau_matched_cuts_tpt_3p.GetBinContent(1)
tau_fake_cuts_tpt_3p_error=tau_matched_cuts_tpt_3p.GetBinError(1)





hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_PoPy.root
hadd Target path: MC.root:/
hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_PoPy.root
hadd Target path: MC.root:/
hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_PoPy.root
hadd Target path: MC.root:/
hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_PoPy.root
hadd Target path: MC.root:/


In [145]:
######### CALCULATION OF RQCD #############

for k in histos:
    rebin=histos[k][0]
    histo=k

    MC_CR_OS=ROOT.TFile.Open("CR_OS/MC.root","READ")
    Data_CR_OS = ROOT.TFile.Open("CR_OS/Data.root","READ")
    MC_CR_SS=ROOT.TFile.Open("CR_SS/MC.root","READ")
    Data_CR_SS = ROOT.TFile.Open("CR_SS/Data.root","READ")

    a=k
 
    mc_cr_os=MC_CR_OS.Get(histo)
    data_cr_os=Data_CR_OS.Get(histo)
    mc_cr_ss=MC_CR_SS.Get(histo)
    data_cr_ss=Data_CR_SS.Get(histo)


    mc_cr_os.SetDirectory(0)
    data_cr_os.SetDirectory(0)
    mc_cr_ss.SetDirectory(0)
    data_cr_ss.SetDirectory(0)

    MC_CR_OS.Close()
    Data_CR_OS.Close()
    MC_CR_SS.Close()
    Data_CR_SS.Close()

    data_cr_os.Add(mc_cr_os,-1)
    data_cr_ss.Add(mc_cr_ss,-1)
    
    
    
    # STORE STAT. UNCERTAINTIES FOR CONTROL REGION

    uncer_cr_os_1prong=ctypes.c_double()
    uncer_cr_ss_1prong=ctypes.c_double()
    uncer_cr_os_3prong=ctypes.c_double()
    uncer_cr_ss_3prong=ctypes.c_double()
    uncer_cr_os=ctypes.c_double()
    uncer_cr_ss=ctypes.c_double()
    
    sigma_rqcd=0
    
    
    if k=="tau_nprongs_cuts_tpt" or k=="tau_nprongs_cuts":
        RQCD=[data_cr_os.IntegralAndError(1,2,uncer_cr_os_1prong)/data_cr_ss.IntegralAndError(1,2,uncer_cr_ss_1prong),data_cr_os.IntegralAndError(3,4,uncer_cr_os_3prong)/data_cr_ss.IntegralAndError(3,4,uncer_cr_ss_3prong),data_cr_os.IntegralAndError(1,4,uncer_cr_os)/data_cr_ss.IntegralAndError(1,4,uncer_cr_ss)]
        
        integral_cr_os_1prong=data_cr_os.Integral(1,2)
        integral_cr_ss_1prong=data_cr_ss.Integral(1,2)
        integral_cr_os_3prong=data_cr_os.Integral(3,4)
        integral_cr_ss_3prong=data_cr_ss.Integral(3,4)
        integral_cr_os=data_cr_os.Integral()
        integral_cr_ss=data_cr_ss.Integral()
        
        sigma_rqcd=[RQCD[0]*((uncer_cr_os_1prong.value/integral_cr_os_1prong)**2+(uncer_cr_ss_1prong.value/integral_cr_ss_1prong)**2)**(0.5),RQCD[1]*((uncer_cr_os_3prong.value/integral_cr_os_3prong)**2+(uncer_cr_ss_3prong.value/integral_cr_ss_3prong)**2)**(0.5),RQCD[2]*((uncer_cr_os.value/integral_cr_os)**2+(uncer_cr_ss.value/integral_cr_ss)**2)**(0.5)]
        
        print(k)
        print("RQCD_1p=",round(RQCD[0],3))
        print("RQCD_3p=",round(RQCD[1],3))
        
        print("Sigma(RQCD_1p)=",round(sigma_rqcd[0],3))
        print("Sigma(RQCD_3p)=",round(sigma_rqcd[1],3))
    else :
        RQCD=data_cr_os.IntegralAndError(1,data_cr_os.GetNbinsX()+1,uncer_cr_os)/data_cr_ss.IntegralAndError(1,data_cr_ss.GetNbinsX()+1,uncer_cr_ss)
        
        integral_cr_os=data_cr_os.Integral(1,data_cr_os.GetNbinsX()+1)
        integral_cr_ss=data_cr_ss.Integral(1,data_cr_os.GetNbinsX()+1)
        
        sigma_rqcd=RQCD*((uncer_cr_os.value/integral_cr_os)**2+(uncer_cr_ss.value/integral_cr_ss)**2)**(0.5)

        
        print(k)
        #print("RQCD=",round(RQCD,3))
        #print("Sigma(RQCD)=",round(sigma_rqcd,3))
        
    # HISTOGRAM FOR SR_OS

    MC_SR_OS=ROOT.TFile.Open("SR_OS/MC.root","READ")
    Data_SR_OS = ROOT.TFile.Open("SR_OS/Data.root","READ")
    MC_SR_SS=ROOT.TFile.Open("SR_SS/MC.root","READ")
    Data_SR_SS = ROOT.TFile.Open("SR_SS/Data.root","READ")


    mc_sr_os=MC_SR_OS.Get(histo)
    data_sr_os=Data_SR_OS.Get(histo)
    mc_sr_ss=MC_SR_SS.Get(histo)
    data_sr_ss=Data_SR_SS.Get(histo)



    mc_sr_os.SetDirectory(0)
    data_sr_os.SetDirectory(0)
    mc_sr_ss.SetDirectory(0)
    data_sr_ss.SetDirectory(0)

    MC_SR_OS.Close()
    Data_SR_OS.Close()
    MC_SR_SS.Close()
    Data_SR_SS.Close()

    ob=data_sr_ss.GetNbinsX()
    s=data_sr_ss.GetXaxis().GetBinLowEdge(1)
    e=data_sr_ss.GetXaxis().GetBinUpEdge(data_sr_ss.GetNbinsX())
    n=5
    
    if rebin=="yy" or rebin=="yn":
        rebining=biner(histos[k][1][0],histos[k][1][1],data_sr_os)
        nb=len(rebining)-1
        #mc_sr_ss=mc_sr_ss.Rebin(nb,"mc_sr_ss",rebining)
        #data_sr_ss=data_sr_ss.Rebin(nb,"data_sr_ss",rebining)
        
        #mc_sr_os=mc_sr_os.Rebin(nb,"mc_sr_os",rebining)
        #data_sr_os=data_sr_os.Rebin(nb,"data_sr_os",rebining)


    MJ_BG=data_sr_ss.Clone()
    MJ_BG.Add(mc_sr_ss,-1)
    
    # SETTING BINS WITH NEGATIVE VALUES TO 0
    for j in range(1,MJ_BG.GetNbinsX()+2):
        if MJ_BG.GetBinContent(j)<0:
            MJ_BG.SetBinContent(j,0.0)
            
    # SCALING MJ_BG SHAPE BY RQCD
    
    if k=="tau_nprongs_cuts_tpt" or k=="tau_nprongs_cuts":
        
        MJ_BG.SetBinContent(2,RQCD[0]*MJ_BG.GetBinContent(2))
        MJ_BG.SetBinContent(4,RQCD[1]*MJ_BG.GetBinContent(4))
        MJ_BG.SetBinError(2,RQCD[0]*MJ_BG.GetBinError(2))
        MJ_BG.SetBinError(4,RQCD[1]*MJ_BG.GetBinError(4))
        
    else : 
        X=MJ_BG.Integral(1,MJ_BG.GetNbinsX()+1)
        if X<0:
            X=0
            #print("SR_SS Yield",X)
            MJ_BG.Scale(RQCD)
        else :
            #print("SR_SS Yield",X)
            MJ_BG.Scale(RQCD)
    #print("\n")
    
    # ADDING CONTROL REGION STAT. UNCERTAINTIES TO MJ BACKGROUND

    if k=="tau_nprongs_cuts" or k=="tau_nprongs_cuts_tpt":
        MJ_BG.SetBinError(2,(MJ_BG.GetBinError(2)**2+(MJ_BG.GetBinContent(2)**2)*(sigma_rqcd[0]/RQCD[0])**2)**(0.5))
        MJ_BG.SetBinError(4,(MJ_BG.GetBinError(2)**2+(MJ_BG.GetBinContent(4)**2)*(sigma_rqcd[1]/RQCD[1])**2)**(0.5))
    else :
        for j in range(1,MJ_BG.GetNbinsX()+2):
            MJ_BG.SetBinError(j,(MJ_BG.GetBinError(j)**2+(MJ_BG.GetBinContent(j)**2)*(sigma_rqcd/RQCD)**2)**(0.5))
            
    # SAVING MJ BG HISTOGRAM

    ##############################
    file=ROOT.TFile.Open("SR_OS/MJ.root","update")
    MJ_BG.Write(a,ROOT.TObject.kOverwrite)
    file.Close()
    ##############################

    #PLOTS

    ##### LOADING INFORMATION ABOUT EACH SAMPLE #####
    
    Data = ROOT.TFile.Open("SR_OS/Data.root","READ")
    Signal=ROOT.TFile.Open("SR_OS/"+signal_sample,"READ")
    VV=ROOT.TFile.Open("SR_OS/VV.root","READ")
    Wjets=ROOT.TFile.Open("SR_OS/Wjets.root","READ")
    Zjets=ROOT.TFile.Open("SR_OS/Zjet.root","READ")
    Ttbar=ROOT.TFile.Open("SR_OS/ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("SR_OS/singletop.root","READ")
    MJ=ROOT.TFile.Open("SR_OS/MJ.root","READ")


    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    mj=MJ.Get(a)
    

    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    mj.SetDirectory(0)

    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    MJ.Close()
    
    ##### SCALING W+JETS
    
    wjets_sf=0.8
    sigma_wjets_sf=0.2
    
    w.Scale(1/wjets_sf)
    
    for j in range(1,w.GetNbinsX()+1):
        w.SetBinError(j,(w.GetBinError(j)**2+(w.GetBinContent(j)**2)*(sigma_wjets_sf/wjets_sf)**2)**0.5)
    
    ####### DECLARING STACK HISTOGRAM AND STAT. UNCERN ONE######
    
    hs = ROOT.THStack("hs","")
    
    error=signal.Clone()
    error.Add(mj,1)
    error.Add(z,1)
    error.Add(vv,1)
    error.Add(tt,1)
    error.Add(t,1)
    error.Add(w,1)
    
    ###### REBINING HISTOGRAMS #######
    
    if rebin=="yy" or rebin=="yn":
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        mj=mj.Rebin(nb,"mj",rebining)
        error=error.Rebin(nb,"error",rebining)
        
    
    ###### CALCULATING THE RATIO MC/DATA #######    
        
    ratio=w.Clone()
    lista = ROOT.TList()
    lista.Add(mj)
    lista.Add(z)
    lista.Add(vv)
    lista.Add(tt)
    lista.Add(t)
    lista.Add(signal)
    ratio.Merge(lista)
    ratio.Divide(data)
    ratio_error=ratio.Clone()
    
    ###### NORMALIZING HISTOGRAM USING VARIABLE BINING #######

    if rebin=="yy":
        hist_list=[signal,data,vv,tt,t,z,w,mj,error]
        normalization(hist_list,histos[k][1][2])
       
    ###### ADDING COLOURS TO THE HISTOGRAMS #######
        
    signal.SetFillColor(ROOT.kAzure+8)
    w.SetFillColor(ROOT.kGreen)
    w.SetLineColor(ROOT.kGreen+1)
    z.SetFillColor(ROOT.kRed)
    z.SetLineColor(ROOT.kRed+1)
    vv.SetFillColor(ROOT.kBlue)
    vv.SetLineColor(ROOT.kBlue+1)
    tt.SetFillColor(ROOT.kYellow)
    tt.SetLineColor(ROOT.kYellow+1)
    t.SetFillColor(ROOT.kCyan)
    t.SetLineColor(ROOT.kCyan+1)
    mj.SetLineColor(ROOT.kMagenta+1)
    mj.SetFillColor(ROOT.kMagenta)
    data.SetLineColor(ROOT.kBlack)
    error.SetFillColorAlpha(ROOT.kBlue,0.5)
    ratio_error.SetFillColorAlpha(ROOT.kBlue,0.5)
    
    ###### CREATING STACK HISTOGRAM #######
    
    hs.Add(vv)
    hs.Add(w)
    hs.Add(z)
    hs.Add(tt)
    hs.Add(t)
    hs.Add(mj)
    hs.Add(signal)

    ##### DRAWING CANVAS #######
    
    canvas = ROOT.TCanvas("canvas")
    canvas.cd()
    
    ##### DRAWING TOP PAD, SETTING MARGINS #######

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
    pad1.SetTopMargin(0.01)
    pad1.SetRightMargin(0.01)
    pad1.SetLeftMargin(0.08)
    pad1.SetBottomMargin(0.0)
    pad1.Draw ()
    pad1.cd ()

    
    
    #### OPTION FOR LOG PLOT #####
    
    if "tracks" in k:
        pad1.SetLogy(1)

    #### DRAWING HISTOGRAMS #####    
        
    data.SetMarkerStyle(ROOT.kFullCircle)
    data.SetMarkerSize(0.7)
    data.Draw("pe")
    error.Draw("E2 same")
    hs.Draw("HIST same")
    error.Draw("E2 same")
    data.Draw("pe same")
    
    if(not production):
    
    ##### DRAWING THE STATISTICS BOX #####
    
        gStyle.SetOptStat(1111111)
    
    ###### SETTING STATS BOX POSITION ######
    
        gStyle.SetStatY(0.97);                
        gStyle.SetStatX(0.20);
        gStyle.SetStatW(0.12);                
        gStyle.SetStatH(0.12);
        
    ###### ADDING A LABEL ACCORDING TO THE CHANNEL ######
    
        data.SetTitle("")
        l=ROOT.TLatex()
        l.SetNDC ()
        l.SetTextSize(0.125)
        l.DrawLatex(0.80,0.85,"Z#rightarrow#tau#tau#rightarrow#tau#mu")
        
    else :
        gStyle.SetOptStat(0)
        data.SetTitle("")
        l=ROOT.TLatex()
        l.SetNDC ()
        l.SetTextSize(0.062)
        l.DrawLatex(0.15,0.85,"Z#rightarrow#tau#tau#rightarrow#tau#mu")
            
        
    
    ###### DRAWING CUT LINES ######
    
    if len(histos[k])>6:
        cut1 = ROOT . TLine (histos[k][6] ,0 ,histos[k][6],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut1 . SetLineColor ( ROOT . kRed+1 )
        cut1 . SetLineWidth (2)
        cut1.SetLineStyle(2)
        
        cut2 = ROOT . TLine (histos[k][7] ,0 ,histos[k][7],data.GetBinContent(data.GetMaximumBin()))
        cut2 . SetLineColor ( ROOT . kRed+1 )
        cut2 . SetLineWidth (2)
        cut2.SetLineStyle(2)
        
        cut1.Draw("same")
        cut2.Draw("same")
    
    data.GetXaxis().Draw()
    pad1.RedrawAxis()
    
    ##### ADJUSTING THE RANGE AND THE TITLE OF THE AXIS #####
    
    if len(histos[k])>5:
        e=histos[k][5]
    pad1.SetTicks(1,1)
    pad1.Update()
    data . GetYaxis (). SetRangeUser (1 ,1.5*data.GetBinContent(data.GetMaximumBin()))
    data . GetXaxis (). SetRangeUser (data.GetBinLowEdge(1) ,e)
    data . GetYaxis (). SetTitle (histos[k][2])
    data . GetXaxis (). SetTitle (histos[k][3])
    data . GetYaxis (). SetTitleSize (0.07)
    data. GetYaxis (). SetTitleOffset (0.59)
    data.GetYaxis().SetLabelSize(0.055)
    data . GetXaxis (). SetTitleSize (0.01)
    
    ###### SETTING THE LEGENDS ON THE PLOT #####

    hs . GetXaxis (). SetRangeUser (s ,e)
    legend = ROOT . TLegend (0.53 ,0.80 ,0.96 ,0.95)
    legend . AddEntry ( vv ,"Diboson")
    legend . AddEntry ( data ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( w ," W+jets ")
    legend . AddEntry ( z ," Z+jet ")
    legend . AddEntry ( t ," Single top ")
    legend . AddEntry ( tt ," ttbar ")
    legend . AddEntry ( mj,"MJ")
    legend . AddEntry ( error,"MC Stat. Error")
    legend.SetTextAlign(22)
    legend.SetNColumns(3)
    gStyle.SetLegendBorderSize(0)
    legend . SetLineWidth (2)
    legend . Draw ()
    hs.SetTitle(a)

    ###### DRAWING PAD2 OF THE CANVAS, DRAWING THE RATIO MC/DATA ######
    
    canvas.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.00 ,1 ,0.35)
    pad2.SetRightMargin(0.01)
    pad2.SetLeftMargin(0.08)
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)
    pad2.Draw ()
    pad2.cd ()
    ratio.SetMarkerStyle(ROOT.kFullCircle)
    ratio.SetMarkerSize(0.7)
    ratio.Draw ("p HIST")
    ratio_error.Draw ("E2 same")
    ratio.Draw ("p hist same")
    ratio.SetStats(0)
    ratio.SetStats(0)
    ratio.SetTitle("")
    ratio . GetYaxis (). SetRangeUser (0.5 ,1.5)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.12)
    ratio . GetYaxis (). SetTitleOffset (0.28)
    ratio . GetXaxis (). SetTitle (histos[k][3])
    ratio . GetXaxis (). SetTitleSize (0.14)
    ratio . GetXaxis (). SetTitleOffset (1.0)
    ratio.GetXaxis().SetLabelSize(0.14)
    ratio.GetYaxis().SetLabelSize(0.08)
    
    
    
    
    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line2 = ROOT . TLine (s ,1.1 ,e,1.1)
    line2 . SetLineColor ( ROOT . kBlack )
    line2 . SetLineWidth (1)
    line2 . SetLineStyle(2)
    line3 = ROOT . TLine (s ,0.9 ,e,0.9)
    line3 . SetLineColor ( ROOT . kBlack )
    line3 . SetLineWidth (1)
    line3 . SetLineStyle(2)
    line4 = ROOT . TLine (s ,1.2 ,e,1.2)
    line4 . SetLineColor ( ROOT . kBlack )
    line4 . SetLineWidth (1)
    line4 . SetLineStyle(2)
    line5 = ROOT . TLine (s ,0.8 ,e,0.8)
    line5 . SetLineColor ( ROOT . kBlack )
    line5 . SetLineWidth (1)
    line5 . SetLineStyle(2)
    line6 = ROOT . TLine (s ,1.3 ,e,1.3)
    line6 . SetLineColor ( ROOT . kBlack )
    line6 . SetLineWidth (1)
    line6 . SetLineStyle(2)
    line7 = ROOT . TLine (s ,0.7 ,e,0.7)
    line7 . SetLineColor ( ROOT . kBlack )
    line7 . SetLineWidth (1)
    line7 . SetLineStyle(2)
    line8 = ROOT . TLine (s ,1.4 ,e,1.4)
    line8 . SetLineColor ( ROOT . kBlack )
    line8 . SetLineWidth (1)
    line8 . SetLineStyle(2)
    line9 = ROOT . TLine (s ,0.6 ,e,0.6)
    line9 . SetLineColor ( ROOT . kBlack )
    line9 . SetLineWidth (1)
    line9 . SetLineStyle(2)
    line . Draw (" same ")
    line2 . Draw (" same ")
    line3 . Draw (" same ")
    line4 . Draw (" same ")
    line5 . Draw (" same ")
    line6 . Draw (" same ")
    line7 . Draw (" same ")
    line8 . Draw (" same ")
    line9 . Draw (" same ")
    line . Draw (" same ")
    
    ##### DRAWING THE CUT LINES IN CASE IT APPLIES #######
    
    if len(histos[k])>6:
        cut1.Draw("same")
        cut2.Draw("same")
    pad2.RedrawAxis()
    
    ###### SAVING THE HISTOGRAMS AS PDF FILES ########
    
    canvas.Print(a+signal_string+".pdf")
    
    ####### PRINTING THE YIELDS FOR EVERY SAMPLE WITH THE STAT ERRORS #######
    if k=="tau_nprongs_cuts_tpt" or k=="tau_nprongs_cuts":
        print("SF 1+3 prongs ----------------------------------------------")
        dic={"data":data,"signal":signal,"vv":vv,"w":w,"z":z,"tt":tt,"t":t,"mj":mj}
        dic_numbers={"data":[0,0],"signal":[0,0],"vv":[0,0],"w":[0,0],"z":[0,0],"tt":[0,0],"t":[0,0],"mj":[0,0]}
        for i in dic:
            error=ctypes.c_double()
            a=dic[i].IntegralAndError(0,1000,error)
            dic_numbers[i][0]=a
            dic_numbers[i][1]=error.value
            
        if "tpt" in k:
            tau_dic={"signal_truth":[tau_truth_cuts_tpt_1p+tau_truth_cuts_tpt_3p,tau_truth_cuts_tpt_1p_error+tau_truth_cuts_tpt_3p_error],"signal_fake":[tau_fake_cuts_tpt_1p+tau_fake_cuts_tpt_3p,tau_fake_cuts_tpt_1p_error+tau_fake_cuts_tpt_3p_error]}
        else:
            tau_dic={"signal_truth":[tau_truth_cuts_1p+tau_truth_cuts_3p,tau_truth_cuts_1p_error+tau_truth_cuts_3p_error],"signal_fake":[tau_fake_cuts_1p+tau_fake_cuts_3p,tau_fake_cuts_1p_error+tau_fake_cuts_3p_error]}
        dic_numbers={**tau_dic,**dic_numbers,"RQCD":[RQCD[2],sigma_rqcd[1]]}
        
        dataframe=pd.DataFrame.from_dict(dic_numbers, orient='index',columns=['Yield', 'Stat. Uncer'])
        display(dataframe.round(decimals=3))
        
    if k=="tau_nprongs_cuts_tpt" or k=="tau_nprongs_cuts":
        print("SF 1 prong ----------------------------------------------")
        dic={"data":data,"signal":signal,"vv":vv,"w":w,"z":z,"tt":tt,"t":t,"mj":mj}
        dic_numbers={"data":[0,0],"signal":[0,0],"vv":[0,0],"w":[0,0],"z":[0,0],"tt":[0,0],"t":[0,0],"mj":[0,0]}
        for i in dic:
            error=ctypes.c_double()
            a=dic[i].IntegralAndError(1,2,error)
            dic_numbers[i][0]=a
            dic_numbers[i][1]=error.value
        if "tpt" in k:
            tau_dic={"signal_truth":[tau_truth_cuts_tpt_1p,tau_truth_cuts_tpt_1p_error],"signal_fake":[tau_fake_cuts_tpt_1p,tau_fake_cuts_tpt_1p_error]}
        else:
            tau_dic={"signal_truth":[tau_truth_cuts_1p,tau_truth_cuts_1p_error],"signal_fake":[tau_fake_cuts_1p,tau_fake_cuts_1p_error]}
        dic_numbers={**tau_dic,**dic_numbers,"RQCD":[RQCD[0],sigma_rqcd[2]]}
            
        dataframe=pd.DataFrame.from_dict(dic_numbers, orient='index',columns=['Yield', 'Stat. Uncer'])
        display(dataframe.round(decimals=3))
            
    if k=="tau_nprongs_cuts_tpt" or k=="tau_nprongs_cuts":
        print("SF 3 prong ----------------------------------------------")
        dic={"data":data,"signal":signal,"vv":vv,"w":w,"z":z,"tt":tt,"t":t,"mj":mj}
        dic_numbers={"data":[0,0],"signal":[0,0],"vv":[0,0],"w":[0,0],"z":[0,0],"tt":[0,0],"t":[0,0],"mj":[0,0]}
        for i in dic:
            error=ctypes.c_double()
            a=dic[i].IntegralAndError(3,4,error)
            dic_numbers[i][0]=a
            dic_numbers[i][1]=error.value
        if "tpt" in k:
            tau_dic={"signal_truth":[tau_truth_cuts_tpt_3p,tau_truth_cuts_tpt_3p_error],"signal_fake":[tau_fake_cuts_tpt_3p,tau_fake_cuts_tpt_3p_error]}
        else:
            tau_dic={"signal_truth":[tau_truth_cuts_3p,tau_truth_cuts_3p_error],"signal_fake":[tau_fake_cuts_3p,tau_fake_cuts_3p_error]}
        dic_numbers={**tau_dic,**dic_numbers,"RQCD":[RQCD[1],sigma_rqcd[1]]}
        
        dataframe=pd.DataFrame.from_dict(dic_numbers, orient='index',columns=['Yield', 'Stat. Uncer'])
        display(dataframe.round(decimals=3))
        print('\n') 
            


lep+nu_pt_topo
RQCD= 1.731
Sigma(RQCD)= 0.015
0.0
200.0
SR_SS Yield 144048.25859451294


tau+nu_pt_topo
RQCD= 1.731
Sigma(RQCD)= 0.015
0.0
200.0
SR_SS Yield 144072.50805282593


tau_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_inside
RQCD= 1.366
Sigma(RQCD)= 0.117
0.0
200.0
SR_SS Yield 274.2109240293503


tau_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_outside
RQCD= 1.362
Sigma(RQCD)= 0.123
0.0
200.0
SR_SS Yield 172.1851856969297


lep_pt0_topo_dphi_btag_iso_rnn_ptmu_omega_mreco
RQCD= 1.364
Sigma(RQCD)= 0.085
0.0
200.0
SR_SS Yield 433.4583230018616


sum_pt_pt_topo
RQCD= 1.731
Sigma(RQCD)= 0.015
0.0
400.0
SR_SS Yield 144274.31124925613


MET_topo_dphi_btag_iso_rnn_ptmu_omega_mreco
RQCD= 1.364
Sigma(RQCD)= 0.085
0.0
300.0
SR_SS Yield 481.37400003231596


Z_pt_cuts_inside
RQCD= 1.366
Sigma(RQCD)= 0.117
0.0
400.0
SR_SS Yield 316.6722252275795


Z_pt_cuts_outside
RQCD= 1.362
Sigma(RQCD)= 0.123
0.0
400.0
SR_SS Yield 234.5754106178356


delta_phi_cuts_butphi
RQCD= 1.202
Sigma(RQCD)= 0.032
SR_SS

,Yield,Stat. Uncer
signal_truth,13594.445,233.707
signal_fake,469.223,32.139
data,15092.000,122.850
signal,14063.668,178.501
vv,295.398,4.740
w,183.215,63.243
z,52.445,3.843
tt,190.431,5.144
t,27.160,1.829
mj,474.136,84.755


SF 1 prong ----------------------------------------------


,Yield,Stat. Uncer
signal_truth,11090.762,158.808
signal_fake,469.223,32.139
data,12565.000,112.094
signal,11559.985,162.027
vv,245.654,4.529
w,168.743,61.459
z,50.230,3.769
tt,158.193,4.698
t,22.320,1.660
mj,422.344,59.801


SF 3 prong ----------------------------------------------


...

,Yield,Stat. Uncer
signal_truth,5950.723,117.045
signal_fake,56.642,11.443
data,6156.000,78.460
signal,6007.365,117.603
vv,130.699,2.269
w,6.463,6.662
z,13.688,1.952
tt,46.370,2.532
t,7.578,0.943
mj,14.265,19.455


SF 3 prong ----------------------------------------------


,Yield,Stat. Uncer
signal_truth,1545.440,59.198
signal_fake,0.000,0.000
data,1444.000,38.000
signal,1545.440,59.198
vv,30.882,1.066
w,0.000,0.000
z,0.116,0.116
tt,14.241,1.394
t,1.374,0.412
mj,0.000,19.455




lep_pt_1p_cuts
RQCD= 1.236
Sigma(RQCD)= 0.094
0.0
200.0
SR_SS Yield 398.7396400710568


lep_pt_3p_cuts
RQCD= 1.656
Sigma(RQCD)= 0.178
0.0
200.0
SR_SS Yield 58.0217493083328


lep_pt_1p_cuts_tpt
RQCD= 1.236
Sigma(RQCD)= 0.094
0.0
200.0
SR_SS Yield 51.93222434818745


lep_pt_3p_cuts_tpt
RQCD= 1.656
Sigma(RQCD)= 0.178
0.0
200.0
SR_SS Yield 9.774362232536077


lep_eta_1p_cuts
RQCD= 1.236
Sigma(RQCD)= 0.094
-2.5
2.5
SR_SS Yield 366.0644953250885


lep_eta_3p_cuts
RQCD= 1.656
Sigma(RQCD)= 0.178
-2.5
2.5
SR_SS Yield 58.40892118215561


lep_eta_1p_cuts_tpt
RQCD= 1.236
Sigma(RQCD)= 0.094
-2.5
2.5
SR_SS Yield 43.349957168102264


lep_eta_3p_cuts_tpt
RQCD= 1.656
Sigma(RQCD)= 0.178
-2.5
2.5
SR_SS Yield 11.266477644443512


tau_pt_1p_cuts_1D
RQCD= 1.236
Sigma(RQCD)= 0.094
0.0
200.0
SR_SS Yield 393.7213699492859


tau_pt_3p_cuts_1D
RQCD= 1.656
Sigma(RQCD)= 0.178
0.0
200.0
SR_SS Yield 61.28358361875871


tau_pt_1p_cuts_tpt_1D
RQCD= 1.236
Sigma(RQCD)= 0.094
0.0
200.0
SR_SS Yield 63.55747791856993




Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file lep+nu_pt_topo_PoPy.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file tau+nu_pt_topo_PoPy.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file tau_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_inside_PoPy.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file tau_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_outside_PoPy.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file lep_pt0_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_PoPy.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file sum_pt_pt_topo_PoPy.pdf has b

In [146]:

def print_yields(first_bin,last_bin,histogram_name,RQCD,sigma_RQCD):
            dic={"data":"Data.root","signal":signal_sample,"vv":"VV.root","w":"Wjets.root","z":"Zjet.root","tt":"ttbar.root","t":"singletop.root","mj":"MJ.root"}
            dic_numbers={"signal_truth":[0,0],"signal_fake":[0,0],"data":[0,0],"signal":[0,0],"vv":[0,0],"w":[0,0],"z":[0,0],"tt":[0,0],"t":[0,0],"mj":[0,0]}
            for i in dic:
                filename = "SR_OS/"+str(dic[i])
                file = ROOT.TFile.Open(filename)
                if i!="mj":
                    histo = file.Get(histogram_name)
                    histo_1d = histo.ProjectionX()
                    error=ctypes.c_double()
                    Yield=histo_1d.IntegralAndError(first_bin,last_bin,error)
                    Error = error.value
                    if i=="w":
                        Yield/=0.8
                        if Yield!=0:
                            Error = Yield*((Error/(Yield*0.8))**2 + ((0.2/0.8))**2)**0.5
                        else :
                            Error = 0
                    dic_numbers[i][0]=Yield
                    dic_numbers[i][1]=Error
                else :
                    histo = file.Get(histogram_name+"_1D")
                    error=ctypes.c_double()
                    Yield=histo.IntegralAndError(first_bin,last_bin,error)
                    dic_numbers[i][0]=Yield
                    dic_numbers[i][1]=error.value
                if i=="signal":
                    histo = file.Get(histogram_name)
                    
                    truth = histo.ProjectionX("truth",2,2)
                    error_truth=ctypes.c_double()
                    Yield_truth=truth.IntegralAndError(first_bin,last_bin,error_truth)
                    Error_truth = error_truth.value
                    dic_numbers["signal_truth"][0]=Yield_truth
                    dic_numbers["signal_truth"][1]=Error_truth
                    
                    #histo = file.Get(histogram_name)
                    
                    fake = histo.ProjectionX("fake",1,1)
                    error_fake=ctypes.c_double()
                    Yield_fake=fake.IntegralAndError(first_bin,last_bin,error_fake)
                    Error_fake = error_fake.value
                    dic_numbers["signal_fake"][0]=Yield_fake
                    dic_numbers["signal_fake"][1]=Error_fake
                file.Close()
                    
            
            dic_numbers={**dic_numbers,"RQCD":[RQCD,sigma_RQCD]}

            dataframe=pd.DataFrame.from_dict(dic_numbers, orient='index',columns=['Yield', 'Stat. Uncer'])
            display(dataframe.round(decimals=3))

In [148]:
print_yields(46,201,"tau_pt_3p_cuts",RQCD,sigma_rqcd)

,Yield,Stat. Uncer
signal_truth,1545.440,59.198
signal_fake,0.000,0.000
data,1444.000,38.000
signal,1545.440,59.198
vv,30.882,1.066
w,0.000,0.000
z,0.116,0.116
tt,14.241,1.394
t,1.374,0.412
mj,19.075,11.910


In [24]:
import os
import shutil
if 'OnlySignal' in os.listdir():
    shutil.rmtree('OnlySignal')
os.mkdir('OnlySignal')
for k in histos:
    rebin=histos[k][0]
    histo=k

    MC_CR_OS=ROOT.TFile.Open("CR_OS/MC.root","READ")
    Data_CR_OS = ROOT.TFile.Open("CR_OS/Data.root","READ")
    MC_CR_SS=ROOT.TFile.Open("CR_SS/MC.root","READ")
    Data_CR_SS = ROOT.TFile.Open("CR_SS/Data.root","READ")

    a=k

    mc_cr_os=MC_CR_OS.Get(histo)
    data_cr_os=Data_CR_OS.Get(histo)
    mc_cr_ss=MC_CR_SS.Get(histo)
    data_cr_ss=Data_CR_SS.Get(histo)

    mc_cr_os.SetDirectory(0)
    data_cr_os.SetDirectory(0)
    mc_cr_ss.SetDirectory(0)
    data_cr_ss.SetDirectory(0)

    MC_CR_OS.Close()
    Data_CR_OS.Close()
    MC_CR_SS.Close()
    Data_CR_SS.Close()

    data_cr_os.Add(mc_cr_os,-1)
    data_cr_ss.Add(mc_cr_ss,-1)

    RQCD=data_cr_os.Integral()/data_cr_ss.Integral()

    # HISTOGRAM FOR SR_OS

    MC_SR_OS=ROOT.TFile.Open("SR_OS/MC.root","READ")
    Data_SR_OS = ROOT.TFile.Open("SR_OS/Data.root","READ")
    MC_SR_SS=ROOT.TFile.Open("SR_SS/MC.root","READ")
    Data_SR_SS = ROOT.TFile.Open("SR_SS/Data.root","READ")

    mc_sr_os=MC_SR_OS.Get(histo)
    data_sr_os=Data_SR_OS.Get(histo)
    mc_sr_ss=MC_SR_SS.Get(histo)
    data_sr_ss=Data_SR_SS.Get(histo)

    mc_sr_os.SetDirectory(0)
    data_sr_os.SetDirectory(0)
    mc_sr_ss.SetDirectory(0)
    data_sr_ss.SetDirectory(0)

    MC_SR_OS.Close()
    Data_SR_OS.Close()
    MC_SR_SS.Close()
    Data_SR_SS.Close()

    ob=data_sr_ss.GetNbinsX()
    s=data_sr_ss.GetXaxis().GetBinLowEdge(1)
    e=data_sr_ss.GetXaxis().GetBinUpEdge(data_sr_ss.GetNbinsX())
    n=5
    if rebin=="yy" or rebin=="yn":
        rebining=biner(s,histos[k][1],e,histos[k][2],histos[k][3])
        nb=len(rebining)-1
        mc_sr_ss=mc_sr_ss.Rebin(nb,"mc_sr_ss",rebining)
        data_sr_ss=data_sr_ss.Rebin(nb,"data_sr_ss",rebining)


    MJ_BG=data_sr_ss.Clone()
    MJ_BG.Add(mc_sr_ss,-1)
    MJ_BG.Scale(RQCD)

    # SAVING MJ BG HISTOGRAM

    ##############################
    file=ROOT.TFile.Open("SR_OS/MJ.root","update")
    MJ_BG.Write(a)
    file.Close()
    ##############################

    #PLOTS

    
    ##### LOADING INFORMATION ABOUT EACH SAMPLE #####
    
    Data = ROOT.TFile.Open("SR_OS/Data.root","READ")
    Signal=ROOT.TFile.Open("SR_OS/Signal.root","READ")
    VV=ROOT.TFile.Open("SR_OS/VV.root","READ")
    Wjets=ROOT.TFile.Open("SR_OS/Wjets.root","READ")
    Zjets=ROOT.TFile.Open("SR_OS/Zjet.root","READ")
    Ttbar=ROOT.TFile.Open("SR_OS/ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("SR_OS/singletop.root","READ")
    MJ=ROOT.TFile.Open("SR_OS/MJ.root","READ")
    Error=ROOT.TFile.Open("SR_OS/Signal.root","READ")

    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    mj=MJ.Get(a)
    error=Error.Get(a)

    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    mj.SetDirectory(0)
    error.SetDirectory(0)

    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    MJ.Close()
    Error.Close()
    
    ##### REBINING HISTOGRAMS #####

    if rebin=="yy" or rebin=="yn":
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        error=error.Rebin(nb,"error",rebining)
    
    ##### IF HISTOGRAMS HAVE VARIABLE BINNING, NORMALIZE IT #####
    
    if rebin=="yy":
        hist_list=[signal,data,vv,tt,t,z,w,mj,error]
        normalization(hist_list)
        
    
    ##### SUBSTRACTING EW BG FROM DATA #####
    
    data_minus_BG=data.Clone()
    data_minus_BG.Add(w,-1)
    data_minus_BG.Add(z,-1)
    data_minus_BG.Add(vv,-1)
    data_minus_BG.Add(tt,-1)
    data_minus_BG.Add(t,-1)
    data_minus_BG.Add(mj,-1)
    
    ##### DEFINING RATIO SIGNAL VS DATA-BG #####
    
    ratio=signal.Clone()
    ratio.Divide(data_minus_BG)
    ratio_error=ratio.Clone()
    
    ##### COLOURING THE HISTOGRAMS ##### 
    
    signal.SetFillColor(ROOT.kAzure+8)
    data_minus_BG.SetLineColor(ROOT.kBlack)
    error.SetFillColorAlpha(ROOT.kBlue,0.5)
    ratio_error.SetFillColorAlpha(ROOT.kBlue,0.5)
    
    ##### DRAWING THE STATISTICS BOX #####
    gStyle.SetOptStat(1111111)

    
    ##### DRAWING CANVAS #######
    canvas = ROOT.TCanvas("canvas")
    canvas.cd()
    
    ##### DRAWING TOP PAD, SETTING MARGINS #######

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
    pad1.SetTopMargin(0.03)
    pad1.SetRightMargin(0.03)
    pad1.SetLeftMargin(0.08)
    pad1.SetBottomMargin(0.0)
    pad1.Draw ()
    pad1.cd ()
    
    ##### SETTING STATS BOX POSITION #######

    gStyle.SetStatY(0.93);                
    gStyle.SetStatX(0.95);
    gStyle.SetStatW(0.1);                
    gStyle.SetStatH(0.1);
    
    #### OPTION FOR LOG PLOT #####
    
    #if "rnn_score" in k:
        #pad1.SetLogy(1)

    #### DRAWING HISTOGRAMS #####    
    
    data_minus_BG.SetMarkerStyle(ROOT.kFullCircle)
    data_minus_BG.SetMarkerSize(0.7)
    data_minus_BG.Draw("pe")
    signal.Draw("HIST same")
    error.Draw("E2 same")
    data_minus_BG.Draw("pe same")
    
    
    #### DRAWING CUT LINES #####  
    
    if len(histos[k])>8:
        cut1 = ROOT . TLine (histos[k][8] ,0 ,histos[k][8],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut1 . SetLineColor ( ROOT . kRed+1 )
        cut1 . SetLineWidth (2)
        cut1.SetLineStyle(2)
        
        cut2 = ROOT . TLine (histos[k][9] ,0 ,histos[k][9],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut2 . SetLineColor ( ROOT . kRed+1 )
        cut2 . SetLineWidth (2)
        cut2.SetLineStyle(2)
        
        cut1.Draw("same")
        cut2.Draw("same")
    
    data_minus_BG.GetXaxis().Draw()
    pad1.RedrawAxis()

    ###### ADDING A LABEL ACCORDING TO THE CHANNEL ######
    
    data_minus_BG.SetTitle("")
    l=ROOT.TLatex()
    l.SetNDC ()
    l.DrawLatex(0.6,0.88,"Z#rightarrow#tau#tau#rightarrow#tau#mu")
    
    ##### ADJUSTING THE RANGE AND THE TITLE OF THE AXIS #####
    
    if len(histos[k])>7:
        e=histos[k][7]

    pad1.SetTicks(1,1)
    pad1.Update()
    data_minus_BG.GetYaxis (). SetRangeUser (1 ,histos[k][6]*data_minus_BG.GetBinContent(data_minus_BG.GetMaximumBin()))
    data_minus_BG.GetXaxis (). SetRangeUser (s ,e)
    data_minus_BG.GetYaxis (). SetTitle (histos[k][4])
    data_minus_BG.GetXaxis (). SetTitle (histos[k][5])

    ###### SETTING THE LEGENDS ON THE PLOT #####
    
    legend = ROOT . TLegend (0.7 ,0.73 ,0.8 ,0.93)
    legend . AddEntry ( data_minus_BG ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( error,"MC Stat. Error")
    gStyle.SetLegendBorderSize(0)
    legend . SetLineWidth (2)
    legend . Draw ()
    
    ###### DRAWING PAD2 OF THE CANVAS, DRAWING THE RATIO MC/DATA ######

    canvas.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.00 ,1 ,0.35)
    pad2.SetRightMargin(0.03)
    pad2.SetLeftMargin(0.08)
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.2)
    pad2.Draw ()
    pad2.cd ()
    ratio.SetMarkerStyle(ROOT.kFullCircle)
    ratio.SetMarkerSize(0.7)
    ratio.Draw ("p HIST")
    ratio_error.Draw ("E2 same")
    ratio.Draw ("p hist same")
    ratio.SetStats(0)
    ratio.SetStats(0)
    ratio.SetTitle("")
    ratio . GetYaxis (). SetRangeUser (0 ,2.0)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.10)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    ratio . GetXaxis (). SetTitle (histos[k][5])
    ratio . GetXaxis (). SetTitleSize (0.09)
    ratio.GetXaxis().SetLabelSize(0.10)
    ratio.GetYaxis().SetLabelSize(0.08)
    
    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line2 = ROOT . TLine (s ,1.2 ,e,1.2)
    line2 . SetLineColor ( ROOT . kBlack )
    line2 . SetLineWidth (1)
    line2 . SetLineStyle(2)
    line3 = ROOT . TLine (s ,0.8 ,e,0.8)
    line3 . SetLineColor ( ROOT . kBlack )
    line3 . SetLineWidth (1)
    line3 . SetLineStyle(2)
    line4 = ROOT . TLine (s ,1.4 ,e,1.4)
    line4 . SetLineColor ( ROOT . kBlack )
    line4 . SetLineWidth (1)
    line4 . SetLineStyle(2)
    line5 = ROOT . TLine (s ,0.6 ,e,0.6)
    line5 . SetLineColor ( ROOT . kBlack )
    line5 . SetLineWidth (1)
    line5 . SetLineStyle(2)
    line6 = ROOT . TLine (s ,1.6 ,e,1.6)
    line6 . SetLineColor ( ROOT . kBlack )
    line6 . SetLineWidth (1)
    line6 . SetLineStyle(2)
    line7 = ROOT . TLine (s ,0.4 ,e,0.4)
    line7 . SetLineColor ( ROOT . kBlack )
    line7 . SetLineWidth (1)
    line7 . SetLineStyle(2)
    line8 = ROOT . TLine (s ,1.8 ,e,1.8)
    line8 . SetLineColor ( ROOT . kBlack )
    line8 . SetLineWidth (1)
    line8 . SetLineStyle(2)
    line9 = ROOT . TLine (s ,0.2 ,e,0.2)
    line9 . SetLineColor ( ROOT . kBlack )
    line9 . SetLineWidth (1)
    line9 . SetLineStyle(2)
    line . Draw (" same ")
    line2 . Draw (" same ")
    line3 . Draw (" same ")
    line4 . Draw (" same ")
    line5 . Draw (" same ")
    line6 . Draw (" same ")
    line7 . Draw (" same ")
    line8 . Draw (" same ")
    line9 . Draw (" same ")
    line . Draw (" same ")
    
    ##### DRAWING THE CUT LINES IN CASE IT APPLIES #######
    
    if len(histos[k])>8:
        cut1.Draw("same")
        cut2.Draw("same")
    pad2.RedrawAxis()
    
    ###### SAVING THE HISTOGRAMS AS PDF FILES ########
    
    os.chdir('OnlySignal/')
    canvas.Print(a+".pdf")
    os.chdir('../')

TypeError: biner() takes 3 positional arguments but 5 were given

In [24]:
import pandas as pd

In [7]:
a={"a":1,"b":2}
b={"c":3,"d":4,"e":5}

c={"a":1,**b,"b":2}

In [8]:
c

{'a': 1, 'c': 3, 'd': 4, 'e': 5, 'b': 2}